In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import re
import requests
import holoviews as hv
import matplotlib.pyplot as plt
import pint
import nd2reader
import tifffile
import scipy.stats
import skimage

In [ ]:
%load_ext autoreload
%autoreload 2
import simulation

In [ ]:
# u = simulation.ureg
class ObjProxy(object):
    def __init__(self, module_name, attr_name):
        self.__module_name = module_name
        self.__attr_name = attr_name

    def __getattr__(self, name):
        return getattr(getattr(globals()[self.__module_name], self.__attr_name), name)


u = ObjProxy("simulation", "ureg")

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
excitation_bins = np.linspace(300, 850, 1000)
emission_bins = excitation_bins

# FP spectra

In [ ]:
# list of urls -> dict of spectra (columns: ex, em, 2p)
# rebin spectra\n",
# rebinned spectra -> 2D spectra (see-saw)

# 2D spectra -> img spectral hypercube ground truth [ex,em,y,x]
# sample line (pass defocus array?) [ex,y,x]
# rebin laser spectrum
# sample line * rebinned laser spectrum [ex,y,x]
# line * sample = emission ground truth [em,y,x]
# emission ground truth + reflectivity * excitation = emission [em,y,x]
# blur/resample? to NA

In [ ]:
spectra_urls = [
    "https://www.fpbase.org/spectra_csv/?q=79,80,158",
    "https://www.fpbase.org/spectra_csv/?q=1451,1450",
    "https://www.fpbase.org/spectra_csv/?q=119,120",
    "https://www.fpbase.org/spectra_csv/?q=121,122",
]

In [ ]:
spectra = {
    name: pd.read_csv(url)
    .rename(columns={f"{name} {kind}": kind for kind in ("ex", "em", "2p")})
    .set_index("wavelength")
    for name, url in spectra_urls.items()
}

# Sample

In [ ]:
# nd2 = nd2reader.ND2Reader("ClpP_mEGFP_100x.nd2")
# nd2._parser._raw_metadata.z_levels = [0]
# sample_img = nd2.get_frame_2D(v=0, c=1)  # [:,500:1000]
# sample_img = image_to_xarray(sample_img, nd2.metadata["pixel_microns"])

In [ ]:
sample_img = tifffile.imread("t000001xy14c2.tif")
scale = 4.25 / 20
sample_img = simulation.image_to_xarray(sample_img, scale)

In [ ]:
sample_img.plot.imshow()

In [ ]:
import skimage.filters

In [ ]:
labels = sample_img.values > skimage.filters.threshold_otsu(sample_img.values)

In [ ]:
num_spectra = 4

In [ ]:
spectra_idx = skimage.morphology.label(labels) % num_spectra + (labels != 0)

# Excitation line

In [ ]:
line_img = simulation.draw_excitation_line(
    1.3 * u.mm,
    np.linspace(0.1, 0.5, 10) * u.um,
    0 * u.um,
    1,
    p_vertical=3,
    p_horizontal=6,
)

In [ ]:
line_img = simulation.draw_excitation_line(
    1.3 * u.mm, 0.3 * u.um, 0 * u.um, 1, p_vertical=3, p_horizontal=6
)

In [ ]:
line_img.plot()

In [ ]:
sample_offset = simulation.offset_xarray(sample_img, line_img, dict(x=500, y=470))

In [ ]:
(sample_offset * line_img).plot(aspect=20, size=5)